# Introduction
## Content:
- Aim: To show how to use BayesRTO.py
1. Apply BayesRTO on Benoit Problem

In [4]:
import numpy as np
import random
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import BayesRTO
import Benoit_Problem

# 1. Benoit Problem
## A. Without Uncertainty

In [5]:
###################################################################################
###########################______GP Initialization______###########################
###################################################################################
'''
Description:
    It does as follows:
    1. Collects samples
    2. Find optimal parameters
    3. Initializes GP.
Arguments:
    `n_sample`      : number of samples to collect your input for GP initialization
    `u_0`           : initial input u (center point of area where you collect  of samples)
    `theta_0`       : initial parameter values for model [parameters for obj fun, parameters for constraint 1, parameters for constraint 2,...]
    `r`             : initial radius of trust region
    `plant_system`  : array with elements of plant objective function and constraints
    `model`         : array with elements of model objective function and constraints
Returns:
    GP_m            : a Gaussian Process Model
'''
BRTO = BayesRTO.Bayesian_RTO()
np.random.seed(42)
random.seed(42)

n_sample            = 4
u_0                 = np.array([4.,-1.])
theta_0             = np.array([1.,1.,1.,1.])
r                   = 1
plant_system        = [Benoit_Problem.Benoit_System_1,
                       Benoit_Problem.con1_system]
model               = [Benoit_Problem.Benoit_Model_1,
                       Benoit_Problem.con1_Model]

theta,GP_m = BRTO.GP_Initialization(n_sample,u_0,theta_0,r,plant_system,model)

#######################################################################################
###########################______Bayesian Optimization______###########################
#######################################################################################
'''
'''
n_iter = 10
for i in range(n_iter):

    # New observation
    d_new = BRTO.observation_trustregion(r,u_0,theta,GP_m)
    # Collect data on new observation
    u_new = u_0 + d_new
    modifier = BRTO.modifier_calc(theta,u_new)

    # Improve GP model using collected data on new observation
    GP_m.add_sample(u_new,modifier)

    # for next iter + data collection
    u_0 = u_new

    print(f"u_new: {u_new}")
    print(f"d_new: {d_new}")
    print(f"mag d_new: {np.linalg.norm(d_new)}")

u_new: [ 3.00562907 -0.89404503]
d_new: [-0.99437093  0.10595497]
mag d_new: 1.0000000000265565
u_new: [ 2.15956118 -1.42712019]
d_new: [-0.84606789 -0.53307515]
mag d_new: 0.9999999999963575
u_new: [ 1.80605383 -2.36255194]
d_new: [-0.35350734 -0.93543175]
mag d_new: 1.0000000000000007
u_new: [ 1.52124686 -2.89970357]
d_new: [-0.28480698 -0.53715164]
mag d_new: 0.6079859331992975
u_new: [ 1.52760343 -2.87813046]
d_new: [0.00635658 0.02157311]
mag d_new: 0.02249011447653471
u_new: [ 1.52760344 -2.87813046]
d_new: [1.33650007e-09 4.64082133e-09]
mag d_new: 4.829436310542298e-09
u_new: [ 1.52760344 -2.87813046]
d_new: [-2.69483093e-12  1.02864554e-15]
mag d_new: 2.694831121611505e-12
u_new: [ 1.52760334 -2.87813014]
d_new: [-9.13921412e-08  3.21492682e-07]
mag d_new: 3.342305609381544e-07
u_new: [ 1.52760497 -2.87814211]
d_new: [ 1.62615372e-06 -1.19757903e-05]
mag d_new: 1.2085691129511082e-05
u_new: [ 1.52760497 -2.87814211]
d_new: [ 4.17024988e-10 -7.11501074e-11]
mag d_new: 4.2305103

## Can find following informations

In [6]:
# 1. Input samples initially created by number n_sample(argument) and used to initialize GP
print(f"Initial Input Sample: \n {BRTO.input_sample}")
print(model[0](theta,u_new,np.array([0,0]),GP_m))
print(model[0](theta,u_new,np.array([-0.5,+0.1]),GP_m))

Initial Input Sample: 
 [[ 5.54070743 -1.42886806]
 [ 4.12378102 -0.7089308 ]
 [ 3.25831462 -1.74163338]
 [ 4.84986638 -0.58699868]]
6.220652775188762
5.626794223407547
